使用torchvision.datasets.ImageFolder加载数据集，并应用数据增强和预处理

In [5]:
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# 数据增强和预处理
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet标准化
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 加载数据集
train_data = datasets.ImageFolder('a5_data/train', transform=train_transform)
val_data = datasets.ImageFolder('a5_data/val', transform=val_transform)

# 创建DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

加载预训练的VGG16模型，并冻结其权重以用作特征提取器

In [6]:
# 加载预训练的VGG16模型
weights_id = torchvision.models.VGG16_Weights.IMAGENET1K_V1
vgg16 = torchvision.models.vgg16(weights=weights_id)

# 冻结模型权重
for param in vgg16.parameters():
    param.requires_grad = False

# 替换分类器
vgg16.classifier = torch.nn.Sequential(
    torch.nn.Linear(25088, 4096),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(4096, 2)  # 二分类任务
)

# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16 = vgg16.to(device)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/laqfire/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 99.2MB/s] 


使用交叉熵损失函数和Adam优化器

In [7]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=0.001)

训练模型并在验证集上评估性能

In [8]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # 前向传播
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # 打印训练损失
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")
        
        # 验证集评估
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_accuracy = correct / total
        print(f"Validation Accuracy: {val_accuracy:.4f}")

# 训练模型
train_model(vgg16, train_loader, val_loader, criterion, optimizer, num_epochs=10)

Epoch 1/10, Loss: 0.7905126045740659
Validation Accuracy: 0.7732
Epoch 2/10, Loss: 0.3805153319121
Validation Accuracy: 0.7564
Epoch 3/10, Loss: 0.3561110910936375
Validation Accuracy: 0.8099
Epoch 4/10, Loss: 0.32751708718674694
Validation Accuracy: 0.8506
Epoch 5/10, Loss: 0.32420311464154306
Validation Accuracy: 0.8474
Epoch 6/10, Loss: 0.3135232103071106
Validation Accuracy: 0.8610
Epoch 7/10, Loss: 0.3093306697274915
Validation Accuracy: 0.8315
Epoch 8/10, Loss: 0.3057387589815244
Validation Accuracy: 0.8339
Epoch 9/10, Loss: 0.29623942566451744
Validation Accuracy: 0.8650
Epoch 10/10, Loss: 0.28215504052182333
Validation Accuracy: 0.8618
